# Collecting Winning Lotto Data

Collect data for past major winners (this is for tickets sold between February 1, 2023 - August 1, 2023)

## Scraping data from OLG Website

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
from dotenv import load_dotenv

In [3]:
url = 'https://about.olg.ca/winners-and-players/ticket-information/where-winning-tickets-were-sold/'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [4]:
print(soup)

<!DOCTYPE html>
<html lang="en-CA" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=5.0, user-scalable=1" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Ontario, Canada, lottery, lotteries, Ontario Lottery and Gaming Corporation, OLGC, Ontario Lottery Corporation, OLC, games, gaming, gambling, responsible gambling, responsible gambling, jackpot, jackpots, winning numbers, government, sport, sports, sports betting, loterie, loteries, Société des loteries et des jeux de l'Ontario, SLJO, Société des loteries de l'Ontario, SLO, jeux, jouer, pari, jeux de loterie sur terminal, gros lot, gros lots, numéros gagnants, gouvernement, sport, sports, pari sportif, Bingo, Instant Bingo, Bingo Instant, Superstar Bingo, 6/49, Lotto 6/49, Lotto Super 7, Super 7, Ontario 49, Lottario, Pick 3, Daily Keno, Ins

In [5]:
table = soup.find_all('table')[0]
print(table)

<table class="table" id="tablepaging"><thead><tr><th class="width-name">Name</th><th class="width-address">Address</th><th class="width-product">Product</th><th class="width-draw">Draw</th><th class="width-value">Value</th></tr></thead><tbody><tr><td class="width-name" data-th="Name"><span>MAC'S CONVENIENCE STORES                </span></td><td class="width-address" data-th="Address"><span>7887 TECUMSEH RD E N8T-1G3 WINDSOR</span></td><td class="width-product" data-th="Product"><span>Lotto Max                               </span></td><td class="width-draw" data-th="Draw"><span>06-Jun-2023                             </span></td><td class="width-value prize" data-th="Value"><span class="inline-block nowrap">$35,000,000.00</span></td></tr><tr><td class="width-name" data-th="Name"><span>SHELL                                   </span></td><td class="width-address" data-th="Address"><span>4685 CENTRAL PKY E L4Z-2E4 MISSISSAUGA</span></td><td class="width-product" data-th="Product"><span>Lo

In [6]:
table_titles = table.find_all('th')
table_titles

[<th class="width-name">Name</th>,
 <th class="width-address">Address</th>,
 <th class="width-product">Product</th>,
 <th class="width-draw">Draw</th>,
 <th class="width-value">Value</th>]

In [7]:
titles = [title.text.strip() for title in table_titles]
titles

['Name', 'Address', 'Product', 'Draw', 'Value']

In [8]:
df = pd.DataFrame(columns=titles)
df

,Name,Address,Product,Draw,Value


In [9]:
column_data = table.find_all('tr')
for row in column_data[1:]:
    row_data = row.find_all('td')
    single_row_data = [data.text.strip() for data in row_data]
    
    length = len(df)
    df.loc[length] = single_row_data

In [10]:
df
# *** NOTE this is for Tickets Sold between February 1, 2023 - August 1, 2023 ***

,Name,Address,Product,Draw,Value
0,MAC'S CONVENIENCE STORES,7887 TECUMSEH RD E N8T-1G3 WINDSOR,Lotto Max,06-Jun-2023,"$35,000,000.00"
1,SHELL,4685 CENTRAL PKY E L4Z-2E4 MISSISSAUGA,Lotto 649,01-Mar-2023,"$5,000,000.00"
2,SHOP RITE CONVENIENCE,5051 BASELINE RD L0E-1R0 SUTTON WEST,Lotto 649,17-May-2023,"$5,000,000.00"
3,SOBEYS #678 - CAMBRIDGE,130 CEDAR ST N1S-1W4 CAMBRIDGE,Lotto 649,01-Feb-2023,"$5,000,000.00"
4,ESSO GAS STATION,1645 NORTH SERVICE RD L2A-6T8 FORT ERIE,Ontario 49,01-Mar-2023,"$2,000,000.00"
...,...,...,...,...,...
775,WESTDALE LOTTERY SERVICES,1151 DUNDAS ST W L5C-1C6 MISSISSAUGA,Daily Keno,26-Mar-2023,"$10,000.00"
776,WILSON MYSTOP CONVENIENCE,1900 WILSON AVE M9M-1A8 NORTH YORK,Lotto 649,22-Jul-2023,"$10,000.00"
777,WILSON MYSTOP CONVENIENCE,1900 WILSON AVE M9M-1A8 NORTH YORK,Pick 4,26-Mar-2023,"$10,000.00"
778,ZEHRS #528 - CONESTOGA,555 DAVENPORT RD N2L-6L2 WATERLOO,CASH IN,31-2367,"$10,000.00"


## Convert Address Data to Coordinates

In [12]:
import re


addresses = list(df.Address)

postal_code_pattern = r'[A-Z]\d[A-Z]-\d[A-Z]\d'

postal_codes = []

for address in addresses:
    match = re.search(postal_code_pattern, address)
    if match:
        postal_codes.append(match.group())

df['Postal_Code'] = postal_codes
print(postal_codes)

['N8T-1G3', 'L4Z-2E4', 'L0E-1R0', 'N1S-1W4', 'L2A-6T8', 'L3C-4W5', 'N3A-1S4', 'N0L-2P0', 'L4T-2H2', 'L4R-4K4', 'N0M-2M0', 'L6A-1C1', 'K0J-1T0', 'L3B-5E4', 'M2J-4Y6', 'L5R-1B2', 'L3R-1A3', 'M1B-3A4', 'L3R-0Y5', 'L7G-5X3', 'L1C-3X1', 'K7S-3G9', 'P2A-2L7', 'K0K-3E0', 'M1E-2N7', 'P3E-4M8', 'K9V-2Y4', 'L5T-2Z6', 'M9W-6K5', 'N8N-4Z4', 'N1R-8N8', 'L9N-0A2', 'P2A-2L7', 'P1L-1B9', 'L1J-2L5', 'L4L-1V8', 'L4C-8W7', 'N2M-3N1', 'L6H-3S7', 'L1J-4H3', 'M1W-3Y1', 'L4K-2P7', 'P2B-1Y2', 'N4G-2H8', 'L1G-3S4', 'L6A-3N9', 'M1N-1V4', 'L9S-2G3', 'L4G-0K3', 'L1S-2C1', 'M1X-1E6', 'K2J-4H9', 'L7L-2Y2', 'N0L-1M0', 'N0A-1E0', 'N1R-4S1', 'M3A-1X8', 'M8V-1J8', 'N2M-2K5', 'M1K-2C1', 'N2A-1B3', 'M8X-1Y8', 'M9V-1C3', 'L3K-5L2', 'M6H-2H6', 'L4W-1V5', 'M1V-1Z6', 'P5E-1S8', 'L1N-7T2', 'K0K-2T0', 'L9B-2M9', 'K9J-4A4', 'M9N-1X9', 'L6V-1C2', 'M1H-2Z9', 'L7J-1R2', 'N7T-5R3', 'L5R-2N3', 'P0R-1B0', 'L5L-3R8', 'L1G-2Y2', 'P0W-1E0', 'N3H-3P6', 'L3R-3V7', 'L6H-6W5', 'M5P-1N4', 'M1W-3G5', 'M2P-2E3', 'N8S-1T1', 'P8N-2P6', 'L4P-2E6'

In [13]:
import geocoder


load_dotenv()
BING_MAPS_KEY = os.getenv('BING_MAPS_KEY')


def batch_geocode(addresses):
    latitudes = []
    longitudes = []
    
    batch_size = 50
    batches = [addresses[i:i + batch_size] for i in range(0, len(addresses), batch_size)]

    for batch in batches:
        g = geocoder.bing(batch, method='batch', key=BING_MAPS_KEY)
        for result in g:
            if result.ok:
                latitudes.append(result.lat)
                longitudes.append(result.lng)
                print(result.latlng)
            else:
                latitudes.append(None)
                longitudes.append(None)

    return latitudes, longitudes

all_latitudes, all_longitudes = batch_geocode(addresses)


[42.313731, -82.932204]
[43.6165, -79.641336]
[44.301238, -79.370775]
[43.352634, -80.328386]
[42.9206366, -78.9671472]
[42.994697, -79.270271]
[43.382435, -80.713769]
[42.603012, -81.60515]
[43.706848, -79.647288]
[44.740107, -79.912902]
[43.055496, -81.006378]
[43.852304, -79.526775]
[45.554451, -77.111455]
[42.974416, -79.235789]
[43.793138, -79.354514]
[43.606682, -79.653705]
[43.814524, -79.348235]
[43.800514, -79.200564]
[43.826189, -79.306286]
[43.632364, -79.877133]
[43.911208, -78.672865]
[45.423192, -76.36876]
[45.343217, -80.016248]
[44.301452, -77.550779]
[43.768628, -79.187363]
[46.451135, -81.004853]
[44.355207, -78.737462]
[43.646441, -79.705061]
[43.720386, -79.600071]
[42.314681, -82.866686]
[43.357432, -80.288152]
[44.074798, -79.482721]
[45.344684, -80.009864]
[45.042181, -79.324271]
[43.891483, -78.895245]
[43.764521, -79.576008]
[43.875826, -79.41472]
[43.431634, -80.484603]
[43.464579, -79.722553]
[43.888587, -78.871011]
[43.814803, -79.322407]
[43.787619, -79.470

In [ ]:
df['Latitude'] = all_latitudes
df['Longitude'] = all_longitudes

In [19]:
def convert_to_float(amount_str):
    amount = amount_str.replace('$', '').replace(',', '')
    return round(float(amount), 2)

# Apply the conversion function to the 'value' column
df['Value'] = df['Value'].apply(convert_to_float)

In [20]:
df

,Name,Address,Product,Draw,Value,Postal_Code,Latitude,Longitude
0,MAC'S CONVENIENCE STORES,7887 TECUMSEH RD E N8T-1G3 WINDSOR,Lotto Max,06-Jun-2023,35000000.00,N8T-1G3,42.31,-82.93
1,SHELL,4685 CENTRAL PKY E L4Z-2E4 MISSISSAUGA,Lotto 649,01-Mar-2023,5000000.00,L4Z-2E4,43.62,-79.64
2,SHOP RITE CONVENIENCE,5051 BASELINE RD L0E-1R0 SUTTON WEST,Lotto 649,17-May-2023,5000000.00,L0E-1R0,44.30,-79.37
3,SOBEYS #678 - CAMBRIDGE,130 CEDAR ST N1S-1W4 CAMBRIDGE,Lotto 649,01-Feb-2023,5000000.00,N1S-1W4,43.35,-80.33
4,ESSO GAS STATION,1645 NORTH SERVICE RD L2A-6T8 FORT ERIE,Ontario 49,01-Mar-2023,2000000.00,L2A-6T8,42.92,-78.97
...,...,...,...,...,...,...,...,...
775,WESTDALE LOTTERY SERVICES,1151 DUNDAS ST W L5C-1C6 MISSISSAUGA,Daily Keno,26-Mar-2023,10000.00,L5C-1C6,43.56,-79.65
776,WILSON MYSTOP CONVENIENCE,1900 WILSON AVE M9M-1A8 NORTH YORK,Lotto 649,22-Jul-2023,10000.00,M9M-1A8,43.72,-79.53
777,WILSON MYSTOP CONVENIENCE,1900 WILSON AVE M9M-1A8 NORTH YORK,Pick 4,26-Mar-2023,10000.00,M9M-1A8,43.72,-79.53
778,ZEHRS #528 - CONESTOGA,555 DAVENPORT RD N2L-6L2 WATERLOO,CASH IN,31-2367,10000.00,N2L-6L2,43.50,-80.53


## Export dataframe to CSV

In [21]:
df.to_csv('lotto_data.csv', index = False)